## pyLDAvis

`pyLDAvis` is a python libarary for interactive topic model visualization.
It is a port of the fabulous [R package](https://github.com/cpsievert/LDAvis>) by Carson Sievert and Kenny Shirley.  They did the hard work of crafting (and testing) an effective visualization. `pyLDAvis` makes it easy to use the visualiziation from Python and, in particualr, IPython notebooks.
